# RAVDESS Song Emotion Recognition with LSTM 
Acoustic features are loaded directly from data directory

In [ ]:
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
!git clone https://github.com/bagustris/ravdess_song.git

Cloning into 'ravdess_song'...
remote: Enumerating objects: 1165, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 1165 (delta 61), reused 32 (delta 12), pack-reused 1057
Receiving objects: 100% (1165/1165), 232.31 MiB | 24.85 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (1030/1030), done.


In [ ]:
cd ravdess_song/

/content/ravdess_song


In [ ]:
# assign hsf to X, lab to y; reshape X for LSTM
X = np.load('data/x.npy', allow_pickle=True)
y = np.load('data/y.npy', allow_pickle=True)

In [ ]:
# reshape x untuk lstm
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [ ]:
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
# functio to build LSTM model
def model_lstm():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [ ]:
# create the model
model = model_lstm()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1, 386)            1544      
_________________________________________________________________
lstm (LSTM)                  (None, 1, 256)            658432    
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1

In [ ]:
# train the Dense model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
57/57 [==============================] - 6s 16ms/step - loss: 1.4176 - accuracy: 0.4418
Epoch 2/100
57/57 [==============================] - 1s 17ms/step - loss: 0.6882 - accuracy: 0.7451
Epoch 3/100
57/57 [==============================] - 1s 17ms/step - loss: 0.4753 - accuracy: 0.8253
Epoch 4/100
57/57 [==============================] - 1s 17ms/step - loss: 0.3855 - accuracy: 0.8495
Epoch 5/100
57/57 [==============================] - 1s 17ms/step - loss: 0.2933 - accuracy: 0.8956
Epoch 6/100
57/57 [==============================] - 1s 17ms/step - loss: 0.3112 - accuracy: 0.8846
Epoch 7/100
57/57 [==============================] - 1s 17ms/step - loss: 0.2247 - accuracy: 0.9121
Epoch 8/100
57/57 [==============================] - 1s 18ms/step - loss: 0.2163 - accuracy: 0.9264
Epoch 9/100
57/57 [==============================] - 1s 18ms/step - loss: 0.1732 - accuracy: 0.9418
Epoch 10/100
57/57 [==============================] - 1s 17ms/step - loss: 0.1910 - accuracy: 0.9286

In [ ]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

7/7 [==============================] - 1s 5ms/step - loss: 0.4030 - accuracy: 0.9118
Loss:  0.4029538333415985 --> Accuracy:  0.9117646813392639
